In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from collections import defaultdict

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO

In [9]:
DATASET_ROOT = '/home/raid_storage/datasets/rosatom'
CSV_PATH = os.path.join(DATASET_ROOT, 'filtered_dataset.csv')

In [10]:
df = pd.read_csv(CSV_PATH, index_col=0)
df = df[df['stage'] == 'test']
df

,filename,x,y,class,stage
7,FRAMES/0/1538/frame0009.bmp,576,313,3,test
8,FRAMES/0/1538/frame0009.bmp,724,509,3,test
13,FRAMES/0/1538/frame0012.bmp,279,326,3,test
14,FRAMES/0/1538/frame0012.bmp,432,503,3,test
16,FRAMES/0/1538/frame0013.bmp,242,331,3,test
...,...,...,...,...,...
33463,FRAMES/2023.10.25/5_498.bmp,493,364,1,test
33536,FRAMES/2023.10.25/5_809.bmp,267,371,8,test
33563,FRAMES/2023.10.25/5_882.bmp,254,95,1,test
33564,FRAMES/2023.10.25/5_882.bmp,250,255,1,test


In [12]:
model_name = './yolo_single_box.pt'
model  = YOLO(model_name)

In [25]:
out = defaultdict(list)
u_images = df['filename'].unique()

for u_image in tqdm(u_images):
    image_path = os.path.join(DATASET_ROOT, u_image)
    results = model.predict(image_path)
    result = results[0]
    
    for box in result.boxes:
        x1, y1, x2, y2 = [int(x) for x in box.xyxy[0].tolist()]
        x = int(x1 + (x2 - x1) / 2)
        y = int(y1 + (y2 - y1) / 2)
        
        conf = round(box.conf[0].item(), 2)
        class_id = int(box.cls[0].item())
        
        out['filename'].append(u_image)
        out['x'].append(x)
        out['y'].append(y)
        out['class'].append(class_id)    
        
    break

  0%|          | 0/908 [00:00<?, ?it/s]
image 1/1 /home/raid_storage/datasets/rosatom/FRAMES/0/1538/frame0009.bmp: 608x960 2 captivitys, 22.3ms
Speed: 2.6ms preprocess, 22.3ms inference, 2.3ms postprocess per image at shape (1, 3, 608, 960)
  0%|          | 0/908 [00:00<?, ?it/s]


In [26]:
result_df = pd.DataFrame(data=out)
result_df.to_csv('multipoint_yolo_predict.csv')

,filename,x,y,class
0,FRAMES/0/1538/frame0009.bmp,725,512,2
1,FRAMES/0/1538/frame0009.bmp,573,313,2
